# Jupyter Notebook: Prioritization of Syphilis Serologies for Investigation (Modernizing the Syphilis Reactor Grid)

## Notes to keep in mind when using this notebook:

* Please ignore the number on the left (that says "In [x]") when running these cells. When we refer to a cell number, look for it in the beginning of the cell body. 


In [ ]:
### TODO BOARD
# edit readme to reflect new contributing: This is a final repository and as such blah blah blah. (keep 2nd paragraph, remove 1st)
# reorganize readme, add beautiful diagram

##### Cell number 1: This cell is for importing python libraries to support the codes that have been used in this Notebook.

In [1]:
# Cell number 1

import sys
import pandas as pd
import numpy as np
import os
import textwrap
import datetime as dt

from collections import defaultdict

# warning suppression
import warnings
warnings.filterwarnings('ignore')

##### Cell number 2: Jupyter Notebook cells output only 1 result by default. Lines 3-4 enable the cell to output all the results as an outcome of the script in the cell. This cell also holds several utility functions for the main algorithm loop. We've also included a convenience function to convert XLSX (excel) files to CSV files.

In [3]:
# Cell number 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Utility Functions

# cleans a column of strings by typecasting to str and stripping whitespace
def cleanstring(df, column):
    df[column] = df[column].astype(str) # converts column items into string
    df[column] = [x.strip() for x in df[column]] # removes any whitespace from the strings generated previously

# these functions return a dataframe with filtered results and are used to reduce redundancy
# probably going to be revisited later

def reactive_nTT(df:pd.DataFrame, col:str, R:str, W:str, U:str, P:str) -> pd.DataFrame:
    return df[(df[col] != R) & (df[col] != W) & (df[col] != U) & (df[col] != P)];

def step3_nonreactive_TT(df:pd.DataFrame, col:str, R:str, W:str, U:str, P:str) -> pd.DataFrame:
    return df[(df[col] == R) | (df[col] == W) | (df[col] == U) | (df[col] == P)];

def step4_TT(df:pd.DataFrame, col:str, filters:list) -> pd.DataFrame:
    return df[(df[col] == filters[0]) | (df[col] == filters[1]) | (df[col] == filters[2]) | (df[col] == filters[3]) | (df[col] == filters[4]) |
              (df[col] == filters[5]) | (df[col] == filters[6]) | (df[col] == filters[7]) | (df[col] == filters[8]) | (df[col] == filters[9])];
######

# converts a given excel file to a csv
def excel2csv(file, name):
    p = pd.read_excel(file)
    p.to_csv(f'{name}.csv', sep=",") 

##### Cell number 3: This is the script to import and read the CSV file. Please copy/paste the address of your file in place of "Please enter your file path and file name here.csv".

##### This file is read into a DataFrame: `df_main`.  It may take a while for the data to be read into `df_main`, so please wait for the cell to be fully executed before proceeding. 

In [4]:
#Cell number 3
df_main = pd.read_csv("data.csv")

##### Cell number 4: Displays the column names of the imported file ( saved as `df_main`) and the first 5 lines of your uploaded dataframe

In [5]:
#Cell number 4
df_main_columns = df_main.columns
df_main.columns = [x.strip() for x in df_main.columns]
df_main.columns
df_main.head()

Index(['Unnamed: 0', 'ID_Profile', 'ID_FieldRecord', 'CD_Disease',
       'DS_DiseaseCategory', 'DateUsedForReport', 'DateUsedForReport_',
       'IN_Morbidity', 'CaseClassification', 'CD_Gender', 'Age', 'AgeUnit',
       'CD_Disposition', 'DS_Disposition', 'DT_Disposition', 'DT_Disposition_',
       'ID_Lab', 'DS_Test', 'DT_Specimen', 'DT_Speciment_',
       'DS_QualitativeResult', 'DS_QuantitativeResult'],
      dtype='object')

##### Cell number 5: This changes the column names for the dataframe. The DataFrame might contain more elements than is required for this algorithm. Those column names can be left as is. Please change the corresponding column names for the following data elements to:
| Column Name | Data Element |
| --- | --- |
| `ID_Profile` | Unique identifier for an individual |
| `Test` | Name of the test (e.g. VDRL, RPR, TP-AB, FTA-ABS) |
| `DT_Specimen` | Date of the above test (specimen collected) |
| `QuantitativeResult` | Quantitative result for test (titer) |
| `QualitativeResult` | Qualitative result for test |
| `CD_Gender` | Gender |
| `Age` | Age (in years) |
| `Disposition` | Disposition assigned |
| `DT_Disposition` | Date the disposition was assigned by the DIS (or the STD program) |

_Note: We require that age be in years. Please handle this yourself or email the maintainers for further assistance._

##### For example, if the output of cell number 4 is: 
`['columnA', 'UniqueIdentifier', 'columnB', 'QualitativeResult','DateOfSpecimen', 'columnC', 'QuantitativeResult', 'NameOfTest', 'columnD','Gender',' Age', 'DS_Disposition','DT_Disposition']`

##### Your code will be:
`df_main.columns = ['columnA', 'ID_Profile', 'columnB', 'QualitativeResult' ,'DT_Specimen', 'columnC', 'QuantitativeResult', 'Test', 'columnD','CD_Gender', 'Age', 'Disposition','DT_Disposition']`

##### Put any column renaming in the cell below:

In [6]:
#Cell number 5
df_main.columns
df_main.rename(columns={'DS_QualitativeResult':'QualitativeResult', 'DS_QuantitativeResult':'QuantitativeResult', 'DS_Test':'Test', 'DS_Disposition':'Disposition'}, inplace=True)
df_main.columns

Index(['Unnamed: 0', 'ID_Profile', 'ID_FieldRecord', 'CD_Disease',
       'DS_DiseaseCategory', 'DateUsedForReport', 'DateUsedForReport_',
       'IN_Morbidity', 'CaseClassification', 'CD_Gender', 'Age', 'AgeUnit',
       'CD_Disposition', 'DS_Disposition', 'DT_Disposition', 'DT_Disposition_',
       'ID_Lab', 'DS_Test', 'DT_Specimen', 'DT_Speciment_',
       'DS_QualitativeResult', 'DS_QuantitativeResult'],
      dtype='object')

Index(['Unnamed: 0', 'ID_Profile', 'ID_FieldRecord', 'CD_Disease',
       'DS_DiseaseCategory', 'DateUsedForReport', 'DateUsedForReport_',
       'IN_Morbidity', 'CaseClassification', 'CD_Gender', 'Age', 'AgeUnit',
       'CD_Disposition', 'Disposition', 'DT_Disposition', 'DT_Disposition_',
       'ID_Lab', 'Test', 'DT_Specimen', 'DT_Speciment_', 'QualitativeResult',
       'QuantitativeResult'],
      dtype='object')

###### Cell number 6: The CSV file does not have date attributes. The script below will assign a python date attribute to this column. The same script can be used for any other date elements. 
###### **The CSV file must contain date the date in the YYYY-MM-DD format. If it is not possible to create it in this format, please let us know.**
###### The final line in this cell creates a new column and assigns the index number of the row to it. This will be used to join on at later stages. If you see output in this cell, please ignore it.

In [7]:
#Cell number 6 

# function to convert strings in column `var` in pandas DataFrame `df` to datetime
def str2date(df, var, dateformat='%Y-%m-%d'):
    cleanstring(df, var)
    
    # The line below can be adjusted to meet unique formats. For example, if the date appears as MM/DD/YYYY, the following
    # format will work: '%m/%d/%Y' Please let us know if you have more complicated date formats.
    # Furthermore, the last parameter can be customized in the function call section to change the specific date time to match your dataset
    # See the function calls section below for a commented out example of this
    df[var] = pd.to_datetime(df[var], format=dateformat) # converts the string to a datetime if it follows YYYY-MM-DD format
    

# function calls
str2date(df_main, 'DT_Specimen')
str2date(df_main, 'DT_Disposition')

## function call example showing how to process datetimes for DT_Specimen if the format is MM/DD/YYYY
# str2date(df_main, 'DT_Specimen', dateformat='%m/%d/%Y')

df_main['S_No'] = df_main.index

###### Cell number 7: The database might default titers as '1:x', the following code removes the '1:' part from it and converts it to an integer. Only use the script below if the quantitative result in the dataframe is not extracted as a number. The script below takes away "1:" from, converts it into a number, and converts nan (No value) to 0. Cell number 7.b will execute the cleaning.

In [8]:
#Cell number 7

def cleannum(a):
    if a == 'nan':
        return 0
    else:
        return (a[2:])

def cleanquant(df, var):
    
    cleanstring(df, var)
    df['quanttest'] = df[var].apply(cleannum)
    df['quanttest'] = df['quanttest'].astype(int)
    

In [9]:
# cell number 7.b
df_main['QuantitativeResult'].value_counts()
# execute function
cleanquant(df_main, 'QuantitativeResult')
df_main['quanttest'].value_counts()

1:1         56191
1:2         43942
1:4         29175
1:8         23066
1:16        18250
1:32        15041
1:64        11334
1:128        9431
1:256        4660
1:512        2057
1:1024        740
1:2048        302
1:4096        124
1:8192         53
1:16384        22
1:999999        1
Name: QuantitativeResult, dtype: int64

0         328849
1          56191
2          43942
4          29175
8          23066
16         18250
32         15041
64         11334
128         9431
256         4660
512         2057
1024         740
2048         302
4096         124
8192          53
16384         22
999999         1
Name: quanttest, dtype: int64

In [10]:
df_main.columns
df_main['QuantitativeResult'].value_counts()

Index(['Unnamed: 0', 'ID_Profile', 'ID_FieldRecord', 'CD_Disease',
       'DS_DiseaseCategory', 'DateUsedForReport', 'DateUsedForReport_',
       'IN_Morbidity', 'CaseClassification', 'CD_Gender', 'Age', 'AgeUnit',
       'CD_Disposition', 'Disposition', 'DT_Disposition', 'DT_Disposition_',
       'ID_Lab', 'Test', 'DT_Specimen', 'DT_Speciment_', 'QualitativeResult',
       'QuantitativeResult', 'S_No', 'quanttest'],
      dtype='object')

nan         328849
1:1          56191
1:2          43942
1:4          29175
1:8          23066
1:16         18250
1:32         15041
1:64         11334
1:128         9431
1:256         4660
1:512         2057
1:1024         740
1:2048         302
1:4096         124
1:8192          53
1:16384         22
1:999999         1
Name: QuantitativeResult, dtype: int64

###### Cell number 8: If the quantitative result or titer was a number in the data extract and cell number 7 was not required, we change the column name to `quanttest`


###### Run Cell number 8 customized to the column headings, labelling the column with titer results (quantitative results) as quanttest, only if cell number 7 was not required

In [11]:
#Cell number 8

#df_main.columns = ['columnA', 'ID_Profile', 'columnB', 'DS_QualitativeResult','DT_Specimen', 'columnC', 'quanttest', 'DS_Test', 'columnD','CD_Gender','Age_clean']


###### Cell number 9: As part of data-wrangling, the following script ensures that the values for `ID_Profile`, `QualitativeResult`, and `Test` do not contain any spaces and are in string format.

###### The data type displayed should show :
| Column Name | Datatype |
| --- | --- |
| `ID_Profile` | `object` |
| `Test` | `object` |
| `DT_Specimen` | `datetime64[ns]` |
| `QuantitativeResult` | `object` |
| `quanttest` | `int32` |
| `CD_Gender` | `object` |
| `Age_clean` | `int32` |
| `Disposition` | `object` |
| `DT_Disposition` | `datetime64[ns]` |

In [12]:
#Cell number 9
dirty_columns = ['ID_Profile', 'QualitativeResult', 'Test', 'CD_Gender', 'Disposition']
for c in dirty_columns:
    cleanstring(df_main, c)

df_main.dtypes

Unnamed: 0                     int64
ID_Profile                    object
ID_FieldRecord                 int64
CD_Disease                     int64
DS_DiseaseCategory            object
DateUsedForReport             object
DateUsedForReport_            object
IN_Morbidity                  object
CaseClassification            object
CD_Gender                     object
Age                           object
AgeUnit                       object
CD_Disposition                object
Disposition                   object
DT_Disposition        datetime64[ns]
DT_Disposition_               object
ID_Lab                        object
Test                          object
DT_Specimen           datetime64[ns]
DT_Speciment_                 object
QualitativeResult             object
QuantitativeResult            object
S_No                           int64
quanttest                      int32
dtype: object

###### Cell number 10: Removes all tests with no dates (they can't be calculated in the algorithm) and reveals how many rows and columns are there in the DataFrame. The following step is not compulsory, but it ensures that all specimen dates have dates else the program will crash.

In [13]:
# Cell number 10
nulldates = df_main[df_main['DT_Specimen'].isnull()] 
beforelen = len(np.unique(nulldates['ID_Profile']))
nulldates_idx = nulldates.index

df_main = df_main.drop(nulldates_idx)
afterlen = len(np.unique(df_main['ID_Profile']))

# ------ output ------
print(f"")
print(f"Number of Null Dates Before Removal: {beforelen}")
print(f"Shape: {df_main.shape}")
print(f"Number of Tests After Null Removal: {afterlen}")


Number of Null Dates Before Removal: 7946
Shape: (534599, 24)
Number of Tests After Null Removal: 113507


###### Cell number 11: Creating a DataFrame for running the algorithm with only the essential columns. Since we select the core data elements, we remove duplicates (same person, same test, same result, same day). The row index is created as a separate column. This `index_no` will be used to join the rest of the data elements for analysis later on
###### We display the number of rows and columns in this DataFrame. There should be 10 columns, the rows depend on how big the dataset is

In [14]:
# Cell number 11

df_main_dd = df_main[['ID_Profile','Test','DT_Specimen','quanttest','QualitativeResult','DT_Disposition']]
df_main_dd = df_main_dd.drop_duplicates()
df_main_dd['index_no'] = df_main_dd.index

df_main_dd = df_main_dd.merge(df_main, left_on = 'index_no', right_on = 'S_No', how = 'left')

# df_main_dd drops invalid tests with no assigned date 
# if there is an error, see below
df_main_dd = df_main_dd[['ID_Profile_x', 'Test_x', 'DT_Specimen_x', 'quanttest_x',
                         'QualitativeResult_x','DT_Disposition_x', 'index_no', 'CD_Gender',
                         'Disposition', 'Age']]


df_main_dd.columns = ['ID_Profile','Test','DT_Specimen','quanttest','QualitativeResult', 'DT_Disposition','index_no','CD_Gender', 'Disposition', 'Age']

#if there is an error:
    #It could be because the column names did not match
    #In a separate cell, please run
    #df_main_dd.dtypes
    #df_main_dd.columns
    
    #under the comment above, please insert how the column name appears in your output in place of these fields above:
    #['ID_Profile_x', 'DS_Test_x', 'DT_Specimen_x', 'quanttest_x',
    #'DS_QualitativeResult_x', 'index_no', 'CD_Gender',
    #'DS_Disposition', 'Age_clean']

In [15]:
# This cell simply prints the dimension of the dataset as well as the column names. 
# Feel free to use this cell to debug the previous cell.
df_main_dd.shape
df_main_dd.columns
df_main_dd.dtypes

(478588, 10)

Index(['ID_Profile', 'Test', 'DT_Specimen', 'quanttest', 'QualitativeResult',
       'DT_Disposition', 'index_no', 'CD_Gender', 'Disposition', 'Age'],
      dtype='object')

ID_Profile                   object
Test                         object
DT_Specimen          datetime64[ns]
quanttest                     int32
QualitativeResult            object
DT_Disposition       datetime64[ns]
index_no                      int64
CD_Gender                    object
Disposition                  object
Age                          object
dtype: object

###### Cell number 12: This is were we define all of our variables. This ensures that we don't have to modify the codes everytime and can just make some adjustments here

In [16]:
# Cell number 12 

#Step 7 allows for a cut-off date for titers since we found that titers can be from a long time ago and should not be compared.
#This can be modified anytime. We define the duration as 'DaysBetweenTests'
DaysBetweenTests = 365

#Step 7 quantifies what should be considered a high titer, given that the previous test is much older
#This can be modified anytime. We define the titer cutoff as 'TiterCutOff'
TiterCutOff = 32

#'Female' maybe coded in many foms, please assign the exact variable as it is in your DataFrame
Female = 'F'

#Step 7 also considers female of reproductive age, to prevent congenital syphilis. We have set the cut-off age as 50
#This can be modified anytime. We define the age as 'FemaleAge'
FemaleAge = 50

#Step 7 was added, in part, to prevent congenital syphilis. For the general population, older titer is considered > 1year
#For females of reproductive age, we consider the duration as 180 days and high titer as more than 1:9
#This can be modified anytime. We define the duration as 'FemaleDaysBetweenTests' and the titer as 'FemaleTiterCutOff'
FemaleDaysBetweenTests = 180
FemaleTiterCutOff = 8

#Step 5: We found that if patient was not previously treated, their titer might not change and can be reached now
#Please assign the exact disposition that the DIS had assigned corresponding to an individual who was infected but not treated
disposition_InfNotTx = 'Infected, Not Treated'
#Please assign the exact disposition that the DIS had assigned corresponding to an individual who was uable to be located
unable_to_locate = 'Unable to Locate'
#Please assign the exact disposition that the DIS had assigned corresponding to an individual who was out of jurisdiction
OOJ = 'Administrative Closure OOJ'

# Please assign values to the corresponding variable as it appears in your dataframe:
#R is for reactive tests
R = 'R'
#W is for weakly positive/reactive tests
W = 'W'
#U is for unknown
U = 'U'
#P is for positive. RAPID tests seem to be assigned this qualitative result
P = 'P'
#N is for negative tests
N = 'N'

#Please assign how each non-Treponemal Test is coded in your dataframe. ONLY these 5 tests will be considered as NTTs.
RPR = 'RPR'
VDRL = 'VDRL'
CSF_VDRL = 'CSF-VDRL'
RPR_CordBlood = 'RPR Cord Blood'
TRUST = 'TRUST'

#Please assign how each Treponemal Test is coded in your dataframe. ONLY these10 tests will be considere as TTs.
FTA_ABS = 'FTA-ABS'
IgG_EIA = 'IgG EIA'
TP_AB = 'TP-AB'
TP_PA = 'TP-PA'
RAPID = 'RAPID'
EIA = 'EIA'
MHATP = 'MHATP'
FTA_IgG = 'FTA-IgG'
CIA = 'CIA (CLIA)'
TPHA = 'TPHA'

#This defines all the Treponemal Tests used for filtering the dataframe in the main loop.
tests = [FTA_ABS, IgG_EIA, TP_AB, TP_PA, RAPID, EIA, MHATP, FTA_IgG, CIA, TPHA]

### Algorithm to Prioritize Reactive Non-Treponemal Tests Reported to Health Departments for Investigating Suspected Cases of Syphilis

![algorithm diagram](algorithm_manuscript_revised.png)

## Step 1: select only reactive Non-Treponemal Tests (NTT)
#### Cell number 13 selects the tests for Step 1 in the algorithm

In [17]:
# cell number 13
dfm_6m_ntt = df_main_dd[(df_main_dd['Test'] == RPR) | (df_main_dd['Test'] == VDRL) | (df_main_dd['Test'] == CSF_VDRL)| (df_main_dd['Test'] == RPR_CordBlood)|(df_main_dd['Test']==TRUST)]
dfm_6m_ntt = dfm_6m_ntt[(dfm_6m_ntt['QualitativeResult'] == R) | (dfm_6m_ntt['QualitativeResult'] == W) | (dfm_6m_ntt['QualitativeResult'] == U)]
dfm_6m_ntt = [group[group['DT_Specimen'] == group['DT_Specimen'].max()] for name , group in dfm_6m_ntt.groupby("ID_Profile")]
dfm_6m_ntt = pd.concat(dfm_6m_ntt)

In [18]:
dfm_6m_ntt['Test'].value_counts()
dfm_6m_ntt.shape
np.unique(dfm_6m_ntt['ID_Profile'])

RPR               117892
VDRL                 993
CSF-VDRL             136
RPR Cord Blood        17
TRUST                  4
Name: Test, dtype: int64

(119042, 10)

array(['1000059', '1000286', '1000370', ..., '999856', '999938', '999940'],
      dtype=object)

#### Cell number 13.1. We want to identify the first test in the latest DT_Disposition (episode of disease). Ideally, we would want to assign a disposition to this first test

In [20]:
#13.1
list1 = np.unique(dfm_6m_ntt['ID_Profile'])
len(list1)
df_first_test = pd.DataFrame(columns=['ID_Profile','Test','DT_Specimen','quanttest','QualitativeResult','CD_Gender', 'Age', 'DT_Disposition', 'index_no'])

for i in list1:
    IncTest = dfm_6m_ntt[dfm_6m_ntt['ID_Profile']==i]
    time_IncTest = IncTest['DT_Disposition']
    time_IncTest = time_IncTest.values[0]
    
    df_all = df_main[df_main['ID_Profile'] == i]
    df_all = df_all[(df_all['Test'] == RPR) | (df_all['Test'] == VDRL) | (df_all['Test'] == CSF_VDRL) | (df_all['Test'] == RPR_CordBlood) | (df_all['Test'] == TRUST)]
    df_sameDT = df_all[df_all['DT_Disposition'] == time_IncTest]
    df_earliestTest = [group[group['DT_Specimen'] == group['DT_Specimen'].min()] for name , group in df_sameDT.groupby("ID_Profile")]
    
    # skips to the next profile if its group doesn't exist
    if not df_earliestTest:
        continue
    
    df_earliestTest = pd.concat(df_earliestTest)
    df_first_test=df_first_test.append(df_earliestTest)   

df_first_test

73332

In [21]:
dfm_6m_ntt = df_first_test

dfm_6m_ntt['Test'].value_counts()
dfm_6m_ntt.shape

RPR         2744
VDRL          16
CSF-VDRL       2
Name: Test, dtype: int64

(2762, 25)

##### Cell number 14: A list (`profile_list`) is created with distinct unique identifiers. This profile list is used to loop in the program. 
##### A single unique identifier (`ID_Profile`) or a subset can be run by creating a list of the subset as `profile_list`. This is especially useful to debug the process and view a specific subset instead of running the entire dataset

In [22]:
# Cell number 14
profile_list = dfm_6m_ntt['ID_Profile'].unique()
profile_list
len(profile_list)

array(['1000059', '1000286', '1000370', ..., '110136', '1101379',
       '1101427'], dtype=object)

1500

##### An example of cell 14.1 to run a fraction of the dataset (for e.g. a cut-off date) instead of the entire dataset. This will be useful to see if there are any bugs or issues.

In [23]:
# Cell number 14.1
profile_list = profile_list[:100]
len(profile_list)

100

In [24]:
#baseline_date = pd.to_datetime('20190318', format='%Y%m%d') #---->change the date within the '' to what you'd like
#baseline_date
#df_main_6m_ntt = df_main_dd[df_main_dd['DT_Specimen'] >= baseline_date]

#df_main_6m_ntt = df_main_dd[(df_main_dd['DS_Test'] == RPR) | (df_main_dd['DS_Test'] == VDRL)|(df_main_dd['DS_Test']==CSF_VDRL)|(df_main_dd['DS_Test']==RPR_CordBlood)]
#df_main_6m_ntt = df_main_6m_ntt[(df_main_6m_ntt['DS_QualitativeResult']==R) | (df_main_6m_ntt['DS_QualitativeResult']==W) | (df_main_6m_ntt['DS_QualitativeResult']==U)]
#df_main_6m_ntt = [group[group['DT_Specimen'] == group['DT_Specimen'].max()] for name , group in df_main_6m_ntt.groupby("ID_Profile")]
#df_main_6m_ntt = pd.concat(df_main_6m_ntt)

#df_main_6m_ntt['DS_Test'].value_counts()
#df_main_6m_ntt.shape

## All of the code for the algorithm is below in cell number 1
### Each loop corresponding to the Step number in the algorithm is assigned that loop number.
### Apart from the disposition assigned in the algorithm, tests are also assigned the exact decision point (for debugging and testing).
### Steps are numbered and commented at every decision node. In case there is an issue with the code, we can print each step to determine where the problem lies.


In [25]:
count = 0
dispo_type = 0
col1 = ['ID_Profile','Test','DT_Specimen','quanttest','QualitativeResult','CD_Gender', 'Age']
col2 = col1.copy()
col2.append('index_no')

# final dataframe which is written to a file at the end
df_complete_merged = pd.DataFrame(columns = ['ID_Profile', 'Test_x', 'DT_Specimen_x', 'quanttest_x',
       'QualitativeResult_x', 'CD_Gender_x', 'Age_x', 'index_no',
       'algo_dispo', 'dispo', 'dis_type', 'Test_y', 'DT_Specimen_y',
       'quanttest_y', 'QualitativeResult_y', 'CD_Gender_y', 'Age_y'])

#1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
for profile_id in profile_list:
    #profile_id
    count = count+1
    sero = 0 
    algo_dispo = 'NA'
    al_dis = 'NA'
    test_tocompare = pd.DataFrame(columns = col1)
    test_incoming = pd.DataFrame(columns = col2)
    
    #Dataframe df_incoming_test isolates the tests for that particular profile_id.
    #We will identify the test that should be assigned a disposition  from this 
    df_incoming_test = dfm_6m_ntt[dfm_6m_ntt['ID_Profile'] == profile_id]

########### This is taking in the reactive nontreponemal tests. We intend to assign a disposition to these tests, whether to administratively close or open for investigation########
    
    #We first select the latest test as 'test_incoming', we will assign a disposition to this test
    if len(df_incoming_test.index) > 1:
        incoming_test = [group[group['DT_Specimen'] == group['DT_Specimen'].max()] for name , group in df_incoming_test.groupby("ID_Profile")]
        test_incoming = pd.concat(incoming_test)
    else:
        test_incoming = df_incoming_test

    #################### We identified test_incoming as THE TEST to assign this disposition ###################


    #time_of_test is the time the specimen was collected for 'test_incoming', it will be used for decision logic below
    time_of_test = pd.to_datetime(test_incoming['DT_Specimen'].values[0])

    
    #!!!!!!!!!! df_match contains the entire dataset; we use this to identify the matches for the test !!!!!!
    df_match = df_main_dd[df_main_dd['ID_Profile'] == profile_id]
    
    #Creating a dataframe 'df_match_all' which has all the tests before 'time_of_test' for each individual ID_Profile
    df_match_all = df_match[df_match['DT_Specimen'] < time_of_test]
    
    #Creating a dataframe 'TT_tocompare' which has all the nonreactive treponemal tests before for each individual ID_Profile
    TT_tocompare = df_match
    TT_tocompare = reactive_nTT(TT_tocompare, 'QualitativeResult', R, W, U, P) 
    TT_tocompare = step4_TT(TT_tocompare, 'Test', tests)
    
    TT_tocompare = TT_tocompare[~(TT_tocompare['DT_Specimen'] > time_of_test)]
    
    #Creating a marker step_2b(REVISIT). It is 1 when 2.b condition is met, otherwise it is 0.
    step_3b = 0
    csf_cord = test_incoming[(test_incoming['Test'] == CSF_VDRL) | (test_incoming['Test'] == RPR_CordBlood)]
    
    #########################Step 222222222222222222222222222222222222222222222
    ######################As per Step 2: current titer CSF or cord blood? #########################
    if csf_cord.empty is False:
        csf_cord = [group[group['quanttest'] == group['quanttest'].max()] for name , group in csf_cord.groupby("ID_Profile")]
        test_incoming = pd.concat(csf_cord)
        test_incoming = test_incoming.iloc[[0]]
        algo_dispo = 'Open: CSF/Cord'
        al_dis = 'Open'
        dispo_type = '2.b'
    
    else:
        
        #333333333333333333333333333333333333333333333333333333333333333333333333333333
        if TT_tocompare.empty is False:
            #If there is a Non-Reactive TT, we identify the latest test
            TT_tocompare = [group[group['DT_Specimen'] == group['DT_Specimen'].max()] for name , group in TT_tocompare.groupby("ID_Profile")]
            TT_tocompare = pd.concat(TT_tocompare)  
            positive_TT = df_match
            
            positive_TT = step3_nonreactive_TT(positive_TT, 'QualitativeResult', R, W, U, P)
            positive_TT = step4_TT(positive_TT, 'Test', tests)
            positive_TT = positive_TT[~(positive_TT['DT_Specimen'] > time_of_test)]
        
            negative_TT = positive_TT[(positive_TT['QualitativeResult'] != R)]
            negative_TT = negative_TT[(positive_TT['QualitativeResult'] != W)]
            negative_TT = negative_TT[(positive_TT['QualitativeResult'] != U)]
            negative_TT = negative_TT[(positive_TT['QualitativeResult'] != P)]
            positive_TT = positive_TT[~(positive_TT['QualitativeResult'].isin(negative_TT['QualitativeResult']))]
        
            if len(positive_TT)>1:    
                if (((pd.to_datetime(time_of_test) - pd.to_datetime(positive_TT['DT_Specimen'].values[0]))/np.timedelta64(1,'D')) <= 14):
                    positive_TT = positive_TT.iloc[[0]]
                    
                
            ##################### Step 3 in the algorithm 3333333333333333333333333333333333333
            ################# Non-reactive TT in last 14 days with no reactive TT reported within this duration
                        
            if (positive_TT.empty is True) & (((pd.to_datetime(time_of_test) - pd.to_datetime(TT_tocompare['DT_Specimen'].values[0]))/np.timedelta64(1,'D')) <= 14): #& (((pd.to_datetime(time_of_test) - pd.to_datetime(TT_tocompare['DT_Specimen'].values[0]))/np.timedelta64(1,'D')) >=0):
                #As per step 3: Non-Reactive Treponemal Test <= 14 Days Prior to Treponemal Test? ############################
                algo_dispo = 'Close: TT_14d'
                al_dis = 'Close'
                dispo_type = '3.b'
                test_tocompare = TT_tocompare.iloc[[0]]
                test_incoming = test_incoming.iloc[[0]]
            # if serology meets this condition then 'step_3b' is assigned  a value '1' and exits the next loop ########
                step_3b = 1
           
        ############################### If step 3 was not met ################################     
        if step_3b == 0:
                       
            # test_incoming := latest test with disposition to assign to

            prior_test_date = df_match_all['DT_Specimen'].max()
            prior_test = df_match_all[df_match_all['DT_Specimen'] == prior_test_date]
            prior_test = step4_TT(prior_test, 'Test', tests)
            prior_test = reactive_nTT(prior_test, 'QualitativeResult', R, W, U, P)
            
            ########################## Step 4 in the algorithm 4444444444444444444444444444444444444444444
            ###################### Penultimate test has a negative treponemal test ###################
            if prior_test.empty is False:
                test_tocompare = prior_test.iloc[[0]]    
                test_incoming = test_incoming.iloc[[0]]
                algo_dispo = 'Open: Pen_N_TT'
                al_dis = 'Open'
                dispo_type = '4.b'
            else:    

                
                ############################ Step 5 in the algorithm 555555555555555555555555555555555555555
                #################### Does patient have Prior Syphilis Non-Treponemal Test? ####################
                
                #'df_match_ntt' creates a dataframe for Non_Treponemal Tests
                df_match_ntt = df_match_all[(df_match_all['Test'] == RPR) | (df_match_all['Test'] == VDRL)]
                
                ## Step 5 logic found in the else statement below near the end of loop close:
                
                if df_match_ntt.empty is False:
                    incoming_test = test_incoming[test_incoming['quanttest']>0]
                    if len(incoming_test) > 0:
                        incoming_test = [group[group['quanttest'] == group['quanttest'].max()] for name , group in incoming_test.groupby("ID_Profile")]
                        incoming_test = pd.concat(incoming_test)
                    
                    ########################## Step 6 in the algorithm 66666666666666666666666666666666666

                    if len(df_match_all) > 1:
                        previouslynotx = [group[group['DT_Specimen'] == group['DT_Specimen'].max()] for name , group in df_match_all.groupby("ID_Profile")]
                        previouslynotx = pd.concat(previouslynotx)
                        previouslynotx = previouslynotx[(previouslynotx['Disposition'] == unable_to_locate) | (previouslynotx['Disposition'] == unable_to_locate) | (previouslynotx['Disposition'] == OOJ) | (previouslynotx['Disposition'] == disposition_InfNotTx)]
                    else:
                        previouslynotx = df_match_all[(df_match_all['Disposition'] == unable_to_locate) | (df_match_all['Disposition'] == unable_to_locate) | (df_match_all['Disposition'] == OOJ) | (df_match_all['Disposition'] == disposition_InfNotTx)]
                    if len(previouslynotx)>1:
                        previouslynotx = [group[group['DT_Specimen'] == group['DT_Specimen'].max()] for name , group in previouslynotx.groupby("ID_Profile")]
                        previouslynotx = pd.concat(previouslynotx)
                   
                    ########################## Previous disposition 'infected, not treated': yes ################
                    if previouslynotx.empty is False:
                        algo_dispo = 'Open: PrevNoTx'
                        al_dis = 'Open'
                        dispo_type = '6.b'
                        previouslynotx = previouslynotx[['ID_Profile','Test','DT_Specimen','quanttest','QualitativeResult']]
                        test_tocompare = previouslynotx.iloc[[0]]
                        test_incoming = test_incoming.iloc[[0]]

                    else:
                        ########################## Step 7 in the algorithm 77777777777777777777777777777777777
                        
                        ## Step 7 logic found in the else statement below near the end of loop close:
                        if incoming_test.empty is False:
                            test_incoming = incoming_test
                            latest_titer = df_match_ntt[df_match_ntt['quanttest'] > 0]
                            
                            #@@@@@@ START: these codes :-
                            ## 1: find previous titer to compare to current serology
                            ## 2: any negative serology prior to current serology to look for serconversion
                            if len(latest_titer) > 0:
                                latest_titer = [group[group['DT_Specimen'] == group['DT_Specimen'].max()] for name , group in latest_titer.groupby("ID_Profile")]
                                # 'latest_titer' identifies the previous titer, before the current reported serology
                                latest_titer = pd.concat(latest_titer)
                                seroconversion = df_match_ntt[df_match_ntt['QualitativeResult'] == N]
                                seroconversion = seroconversion[seroconversion['DT_Specimen']>=latest_titer['DT_Specimen'].values[0]]
                            else:
                                last_titer = df_match_ntt[df_match_ntt['QualitativeResult']!=N]

                                if len(last_titer) > 0:
                                    last_titer = [group[group['DT_Specimen'] == group['DT_Specimen'].max()] for name , group in last_titer.groupby("ID_Profile")]
                                    last_titer = pd.concat(last_titer)
                                    seroconversion = df_match_ntt[df_match_ntt['QualitativeResult'] == N]
                                    seroconversion = seroconversion[seroconversion['DT_Specimen'] >= last_titer['DT_Specimen'].values[0]]
                                else:
                                    seroconversion = df_match_ntt[df_match_ntt['QualitativeResult'] == N]

                            if len(seroconversion) > 1:
                                seroconversion = [group[group['DT_Specimen'] == group['DT_Specimen'].max()] for name , group in seroconversion.groupby("ID_Profile")]
                                seroconversion = pd.concat(seroconversion)
                                seroconversion = seroconversion.iloc[[0]]
                            #@@@@@@ END: these codes :-
                            ## 1: find previous titer to compare to current serology
                            ## 2: any negative serology prior to current serology to look for serconversion

                            ########################## Step 8 in the algorithm 8888888888888888888888888888888888888
                            if latest_titer.empty is False:
                                ############## (Current titer>1:32 AND previous titer >1 year) #####################
                                ##############                           OR                    #####################
                                ########### (Female < 50y AND titer > 1:8 AND previous titer > 6m) #################
                                
                                duration = (pd.to_datetime(time_of_test) - pd.to_datetime(latest_titer['DT_Specimen'].values[0]))/np.timedelta64(1,'D')
                                sero2 = (pd.to_datetime(seroconversion['DT_Specimen']) - pd.to_datetime(latest_titer['DT_Specimen'].values[0]))/np.timedelta64(1,'D')

                                if (test_incoming['CD_Gender'].values[0]==Female) & (duration >= FemaleDaysBetweenTests) & (int(test_incoming['Age'].values[0]) < FemaleAge) & (test_incoming['quanttest'].values[0] > FemaleTiterCutOff):
                                    algo_dispo = 'Open: ModCrit'
                                    al_dis = 'Open'
                                    dispo_type = '8.b2'
                                    test_tocompare = latest_titer.iloc[[0]]    
                                    test_incoming = test_incoming.iloc[[0]]
                                elif (duration >= DaysBetweenTests) & (test_incoming['quanttest'].values[0]>TiterCutOff):
                                    algo_dispo = 'Open: ModCrit'
                                    al_dis = 'Open'
                                    dispo_type = '8.b1'
                                    test_tocompare = latest_titer.iloc[[0]]
                                    test_incoming = test_incoming.iloc[[0]]
                                
                                else:
                                    ##################### Step 9 in the algorithm 999999999999999999999999999999999999
                                    ##################### >=4-fold Titer Increase: Yes ###############################
                            
                                    if test_incoming['quanttest'].values[0] >= ((latest_titer['quanttest'].values[0]*2)*2):
                                        algo_dispo = 'Open: 4f'
                                        al_dis = 'Open'
                                        dispo_type = '9.b1'
                                        test_tocompare = latest_titer.iloc[[0]]
                                        test_incoming = test_incoming.iloc[[0]]
                                    else:
                                        ################# Step 10 in the algorithm 1010101010101010101010101010101010
                                        
                                        if (test_incoming['QualitativeResult'].values[0]==R) & seroconversion.empty is False:
                                            ################# Check for serocoversion ###################################
                                            algo_dispo = 'Open: SeroConv'
                                            al_dis = 'Open'
                                            dispo_type = '10.b1'
                                            test_tocompare = seroconversion
                                            test_incoming = test_incoming.iloc[[0]]

                                        else:
                                            ################# No seroconversion, close: no 4-fold increase ###############
                                            algo_dispo = 'Close: 4f'
                                            al_dis = 'Close'
                                            dispo_type = '10.a1'
                                            test_incoming = test_incoming[test_incoming['QualitativeResult'] == R]
                                            test_incoming = test_incoming.iloc[[0]]
                                            test_tocompare = latest_titer.iloc[[0]]
                                        ############ Step 10 close loop 1010101010101010101010101010101010101010
                                    ################ Step 9 close loop 999999999999999999999999999999999999999999
                            
                            #################### if there were no titers reported previously#####################
                            else:
                                
                                ################## if serconversion is present ############################
                                if (test_incoming['QualitativeResult'].values[0]==R) & seroconversion.empty is False:
                                    #As per step Non-Treponemal Seroconversion: Yes **********************
                                    algo_dispo = 'Open: SeroConv'
                                    al_dis = 'Open'
                                    dispo_type = '10.b2'
                                    test_tocompare = seroconversion.iloc[[0]]
                                    test_incoming = test_incoming.iloc[[0]]
                                    
                                ################## no seroconversion, no previous titer ############################   
                                elif last_titer.empty is False:
                                    algo_dispo = 'Open: NoPrevTi'
                                    al_dis = 'Open'
                                    dispo_type = '9.b2'
                                    test_tocompare = last_titer.iloc[[0]]
                                    test_incoming = test_incoming.iloc[[0]]
                                    test_incoming = test_incoming.iloc[[0]]
                                
                                ################## does not meet 4-fold increase criteria ##########################
                                else:
                                    
                                    algo_dispo = 'Close: 4f'
                                    al_dis = 'Close'
                                    dispo_type = '10.a2'
                                    test_incoming = test_incoming[test_incoming['QualitativeResult'] == R]
                                    test_incoming = test_incoming.iloc[[0]]
                                    test_tocompare = latest_titer.iloc[[0]]
                                    
                            ####################### Step 8 close loop 888888888888888888888888888888888888888888
                        ######################## Step 7 in the algorithm#############################
                        #######################  Quantitative titer reported on current serology? ####################
                        else:
                            algo_dispo = 'Open: NoC_NTT'
                            al_dis = 'Open'
                            dispo_type = '7.a'
                            test_incoming = test_incoming.iloc[[0]]

                        ########################### Step 7 close loop 77777777777777777777777777777777777777777
                    ############################### Step 6 close loop 66666666666666666666666666666666666666666
                ######################## Step 5 in the algorithm#############################
                #######################  Does patient have Prior Syphilis Non-Treponemal Test? ####################
                
                else:
                    algo_dispo = 'Open: NoM'
                    al_dis = 'Open'
                    dispo_type = '5.a'
                    test_incoming = [group[group['quanttest'] == group['quanttest'].max()] for name , group in test_incoming.groupby("ID_Profile")]
                    test_incoming = pd.concat(test_incoming)
                    test_incoming = test_incoming.iloc[[0]]

                #################################### Step 5 close loop 5555555555555555555555555555555555555555
            ######################################## Step 4 close loop 4444444444444444444444444444444444444444
        ############################################ Step 3 close loop 3333333333333333333333333333333333333333
    ################################################ Step 2 close loop 2222222222222222222222222222222222222222
    test_incoming['algo_dispo'] = algo_dispo
    test_incoming['dispo'] = al_dis
    test_incoming['dis_type'] = dispo_type
    df_complete = pd.merge(test_incoming, test_tocompare, on='ID_Profile',how='left')
    df_complete_merged = df_complete_merged.append(df_complete)
#1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
#print(count)           
#df_complete_merged

##### Cell number 16: The script below assigns 1 column for index number for the test. This index number is inherited from the original file df_main

In [26]:
df_complete_merged['test_index'] = df_complete_merged['index_no'].astype(str) + df_complete_merged['index_no_x'].astype(str)
df_complete_merged['test_index'] = df_complete_merged['test_index'].map(lambda x: x.lstrip('nan').rstrip('nan'))
df_complete_merged=df_complete_merged.drop(columns=['index_no', 'index_no_x'])

str2date(df_complete_merged, 'DT_Specimen_x')
str2date(df_complete_merged, 'DT_Specimen_y')

df_complete_merged['day_diff'] = pd.to_datetime(df_complete_merged['DT_Specimen_x']) - pd.to_datetime(df_complete_merged['DT_Specimen_y'])
df_complete_merged['day_value'] = df_complete_merged['day_diff']/np.timedelta64(1,'D')
#df_complete_merged

In [27]:
df_complete_merged['dispo'].value_counts()
df_complete_merged['algo_dispo'].value_counts()
df_complete_merged['dis_type'].value_counts()

Open     56
Close    44
Name: dispo, dtype: int64

Open: NoM         50
Close: 4f         41
Open: 4f           3
Close: TT_14d      3
Open: PrevNoTx     1
Open: NoC_NTT      1
Open: SeroConv     1
Name: algo_dispo, dtype: int64

5.a      50
10.a1    41
9.b1      3
3.b       3
6.b       1
7.a       1
10.b2     1
Name: dis_type, dtype: int64

##### Cell number 17 creates a CSV file for this dataset. Please replace 'ResultOfAlgorithm.csv' with a file path and name of your choice.

In [51]:
# Cell number 17
df_complete_merged.to_csv(r'ResultOfAlgorithm.csv', index=False)

##### Cell number 18: We join the dataset generated by the algorithm script (`df_complete_merged`) on the main dataset (`df_main`). `test_index` and `S_No` are the same index inherited from `df_main`. We conduct a left join on this index number of both datasets

In [28]:
# Cell number 18
df_comp_merged_co = df_complete_merged.copy()
df_main_co = df_main.copy()

df_comp_merged_co['S_No'] = df_comp_merged_co['S_No'].astype(str)
df_main_co['S_No'] = df_main_co['S_No'].astype(str)
df_joined = df_comp_merged_co.merge(df_main_co, left_on='test_index',right_on = 'S_No', how='left')

##### Cell number 19 creates a CSV file for this dataset. Please replace 'AlgorithmMerged.csv' with a file path and name of your choice.

In [40]:
# Cell number 19
df_joined.to_csv(r'AlgorithmMerged.csv', index=False)

## This is the end of the script for the algorithm

In [29]:
df_complete_merged['dispo'].value_counts()
df_complete_merged['algo_dispo'].value_counts()
df_complete_merged['dis_type'].value_counts()

Open     56
Close    44
Name: dispo, dtype: int64

Open: NoM         50
Close: 4f         41
Open: 4f           3
Close: TT_14d      3
Open: PrevNoTx     1
Open: NoC_NTT      1
Open: SeroConv     1
Name: algo_dispo, dtype: int64

5.a      50
10.a1    41
9.b1      3
3.b       3
6.b       1
7.a       1
10.b2     1
Name: dis_type, dtype: int64

In [30]:
df_complete_merged.groupby(['Disposition','algo_dispo'])["ID_Profile"].count()

Disposition                                  algo_dispo    
Administrative Closure OOJ                   Open: NoM          1
Administrative Closure per Reactor Grid      Open: NoM         23
Infected, Brought to Non-Standard Treatment  Open: NoM          1
Infected, Brought to Treatment               Open: NoC_NTT      1
                                             Open: NoM          5
Infected, Not Treated                        Open: NoM          1
                                             Open: PrevNoTx     1
Located, Refused Examination                 Open: NoM          1
Not Infected                                 Open: NoM          1
Previously Treated for this Infection        Open: NoM         17
Name: ID_Profile, dtype: int64